In [1]:
import numpy as np
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pandas as pd
import joblib

In [2]:
texts = pd.read_parquet('datos.parquet')

In [3]:
texts

,El gato está durmiendo en el sofá
0,El perro juega en el parque
1,La comida está lista en la mesa
2,El sol brilla en el cielo azul
3,Los pájaros cantan en el árbol
4,El día amaneció soleado y los pájaros cantaban...
...,...
239,Paco comió una rebanada de pan con manteca
240,Si llueve me pongo la capucha
241,Mi papá es muy bueno jugando al básquet
242,"En un día de sol en el fondo del mar, el pulpo..."


In [53]:
# Tokenizar el texto
tokenizer = Tokenizer()
#tokenizer = joblib.load("tokenizer.pkl")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Padding para igualar la longitud de las secuencias
#maxlen = max(len(seq) for seq in sequences)
maxlen = 100
padded_sequences = pad_sequences(sequences, maxlen=maxlen)

# Datos de entrada y salida
x_train = padded_sequences[:, :-1]  # Todas las columnas excepto la última
y_train = padded_sequences[:, -1]   # Última columna

In [56]:
# Tamaño del vocabulario
vocab_size = len(tokenizer.word_index) + 1

# Modelo de lenguaje
embedding_dim = 50
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=maxlen-1),
    LSTM(100),
    Dense(vocab_size, activation='softmax')
])

c:\Users\marko\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [45]:
model = load_model('modelo_text.h5')

In [57]:
model.summary()

# Compilar y entrenar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100)

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 1.9455
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 1.9097
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 1.8705
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 1.8230
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 1.7610
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 1.6747
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 1.5462
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 1.0000 - loss: 1.3400
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 0.9976
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 1.0000 - loss: 0.5569
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 0.2701
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - 

In [61]:
def generate_text(model, tokenizer, input_text, maxlen, num_words):
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=maxlen-1)

    generated_sentence = input_text
    for _ in range(num_words):
        predicted_probabilities = model.predict(input_seq, verbose=0)
        predicted_word_idx = np.random.choice(len(predicted_probabilities[0]), p=predicted_probabilities[0])
        generated_word = tokenizer.index_word.get(predicted_word_idx, "")

        if generated_word:
            generated_sentence += " " + generated_word
            input_seq = np.append(input_seq[:, 1:], predicted_word_idx)
            input_seq = input_seq.reshape(1, -1)

    return generated_sentence

input_text = "la casa esta pintada de rosa"
generated_text = generate_text(model, tokenizer, input_text, maxlen, num_words=1)
print("Texto generado:", generated_text)

Texto generado: la casa esta pintada de rosa sofá


In [62]:
# Guardar el modelo existente
model.save('modelo_text1.h5')

In [63]:
joblib.dump(tokenizer, 'tokenizer.pkl')

['tokenizer.pkl']